# The Schnorr Protocol

**Module 09** | 09-commitments-sigma-protocols

*Interactive Schnorr, completeness/soundness/ZK*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **the schnorr protocol**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [Sigma Protocols](09c-sigma-protocols-intuition.ipynb).
- Concepts and notation introduced in the previous notebook.

## Protocol Description

In [ ]:
# Prover knows x such that h = g^x
# 1. Prover picks random r, sends a = g^r
# 2. Verifier sends random challenge e
# 3. Prover sends z = r + e*x
# Verify: g^z == a * h^e
# TODO: implement the interactive protocol

## Completeness

In [ ]:
# An honest prover always convinces the verifier
# g^z = g^(r + ex) = g^r * g^(ex) = a * h^e
# TODO: verify completeness with a concrete example

## Soundness and Zero-Knowledge

In [ ]:
# Special soundness: two transcripts -> extract x
# ZK: simulator picks z, e first, computes a = g^z / h^e
# TODO: implement the simulator

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **the schnorr protocol**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [The Fiat-Shamir Transform](09e-fiat-shamir-transform.ipynb)